In [1]:
import numpy as np
from astropy.coordinates import Angle
from astropy import units
import spec_calcs

In [2]:
max_decorr = 0.1
frequency = (250. * 1e6 * units.Hz)
max_baseline_m = (870 * units.m)

The actual math has been moved to the spec_calc module so it can be reused in other code

In [3]:
params = spec_calcs.averaging_params(max_decorr=max_decorr, frequency=frequency, baseline=max_baseline_m,
                                     corr_FoV=Angle(90., units.degree), verbose=True)

Max resolution:
4.73844arcmin
Max integration time required to keep the decorrelation due to time integrating under max_decorr on
the longest baselines:
1.8902011330061352 s
Max channel width to keep the decorrelation due to channel width under max_decorr for a 90.0 degree
correlator FoV on the longest baselines:
34.45890321839081 kHz
Max integration time to keep the decorrelation due to time integrating after fringe stopping under
max_decorr for a 90.0 degree correlator FoV on the longest baselines:
3.6999943247235914 s


In [4]:
params = spec_calcs.averaging_params(max_decorr=max_decorr, frequency=frequency, baseline=max_baseline_m,
                                     corr_FoV=Angle(10., units.degree), verbose=True)

Max resolution:
4.73844arcmin
Max integration time required to keep the decorrelation due to time integrating under max_decorr on
the longest baselines:
1.8902011330061352 s
Max channel width to keep the decorrelation due to channel width under max_decorr for a 10.0 degree
correlator FoV on the longest baselines:
198.44091473557216 kHz
Max integration time to keep the decorrelation due to time integrating after fringe stopping under
max_decorr for a 10.0 degree correlator FoV on the longest baselines:
21.307418105017184 s


Now invert these equations to calculate the decorrelation caused by some suggested correlator settings

In [5]:
# This is the max number of channels that Jack can do 'easily'
n_channels = 4096
corr_chan_width = (250 * units.MHz) / n_channels
# Since we will integrate longer after fringe stopping, there's no need to integrate more here
corr_integration_time = .1 * units.s
# after fringe stopping, we can sum in time to longer integrations
corr_post_fringe_stop_int_time = 10 * units.s
print('proposed channel width: ', corr_chan_width.to(units.kHz))
print('proposed pre fringe stopping integration time: ', corr_integration_time.to(units.ms))
print('proposed post fringe stopping integration time: ', corr_post_fringe_stop_int_time)

proposed channel width:  61.03515625 kHz
proposed pre fringe stopping integration time:  100.0 ms
proposed post fringe stopping integration time:  10.0 s


In [6]:
decorrelations = spec_calcs.decorrelations(frequency=frequency, baseline=max_baseline_m,
                                           corr_FoV=Angle(90., units.degree),
                                           n_channels=n_channels, chan_width=corr_chan_width,
                                           integration_time=corr_integration_time,
                                           fringe_stop_int_time=corr_post_fringe_stop_int_time,
                                           verbose=True)

Decorrelation fraction due to integration time on the longest baseline
0.005290442284359557
Longest E-W baseline with less than 10.0% decorrelation due to integration time
16444.74466458891 m
Decorrelation due to channel width for a 90.0 degree correlator FoV for the longest baseline
0.17712448902733904
Longest baseline for a 90.0 degree correlator FoV with less than 10.0% decorrelation due to channel
width
491.17996318720003 m
Correlator FoV with less than 10.0% decorrelation due to channel width for the longest baseline
34d22m21.9391s
Decorrelation due to time integrating after fringe stopping for a 90.0 degree correlator FoV for the
longest baseline
0.27027068482725447
Correlator FoV with less than 10.0% decorrelation just due to time integrating after fringe stopping
for the longest baseline
21d42m56.0962s

Decorrelation factors actually need to be combined!

Net decorrelation before fringe stopping due to integration time and channel width for 90.0 degree
correlator FoV for the lo

In [7]:
decorrelations = spec_calcs.decorrelations(frequency=frequency, baseline=max_baseline_m,
                                           corr_FoV=Angle(10., units.degree),
                                           n_channels=n_channels, chan_width=corr_chan_width,
                                           integration_time=corr_integration_time,
                                           fringe_stop_int_time=corr_post_fringe_stop_int_time,
                                           verbose=True)

Decorrelation fraction due to integration time on the longest baseline
0.005290442284359557
Longest E-W baseline with less than 10.0% decorrelation due to integration time
16444.74466458891 m
Decorrelation due to channel width for a 10.0 degree correlator FoV for the longest baseline
0.030757344739783626
Longest baseline for a 10.0 degree correlator FoV with less than 10.0% decorrelation due to channel
width
2828.5926739140245 m
Correlator FoV with less than 10.0% decorrelation due to channel width for the longest baseline
34d22m21.9391s
Decorrelation due to time integrating after fringe stopping for a 10.0 degree correlator FoV for the
longest baseline
0.046932011897046017
Correlator FoV with less than 10.0% decorrelation just due to time integrating after fringe stopping
for the longest baseline
21d42m56.0962s

Decorrelation factors actually need to be combined!

Net decorrelation before fringe stopping due to integration time and channel width for 10.0 degree
correlator FoV for the 

Data rate calculations:

In [8]:
n_antennas = 350
channels_to_keep = n_channels * 3/4
obs_hrs_per_day=(12 * units.hour / units.day)
days_per_season=(120 * units.day)

In [9]:
rates = spec_calcs.data_rates(integration_time=corr_integration_time,
                              fringe_stop_int_time=corr_post_fringe_stop_int_time,
                              n_antennas=n_antennas, n_channels=channels_to_keep)

In [10]:
print('Naive (correlator internal):')
print('data rate: ', rates[0].to(units.Gbyte/units.s))
print('channel width: ', corr_chan_width.to(units.kHz))
print('integration time: ', corr_integration_time)

Naive (correlator internal):
data rate:  120.76646400000001 Gbyte / s
channel width:  61.03515625 kHz
integration time:  0.1 s


In [11]:
print('post fringe stopping:')
print('data rate: ', rates[1].to(units.Gbyte/units.s))
season_data_vol = (rates[1].to(units.Pbyte / units.hour) * obs_hrs_per_day * days_per_season)
print('data volume per season: ', season_data_vol)
print('channel width: ', corr_chan_width.to(units.kHz))
print('integration time: ', corr_post_fringe_stop_int_time)

post fringe stopping:
data rate:  1.2076646400000002 Gbyte / s
data volume per season:  6.26053349376 Pbyte
channel width:  61.03515625 kHz
integration time:  10.0 s


In [12]:
daily_data_vol = (rates[1].to(units.Tbyte / units.hour) * obs_hrs_per_day)
print('data volume per day:',daily_data_vol)

data volume per day: 52.171112448 Tbyte / d


In [13]:
bda_factor = 0.1

In [14]:
bda_daily_data_vol = bda_factor * (rates[1].to(units.Tbyte / units.hour) * obs_hrs_per_day)
bda_season_data_vol = bda_factor * (rates[1].to(units.Pbyte / units.hour) * obs_hrs_per_day * days_per_season)
print('post BDA data volume per day:',bda_daily_data_vol)
print('post BDA data volume per season: ', bda_season_data_vol)

post BDA data volume per day: 5.217111244800001 Tbyte / d
post BDA data volume per season:  0.626053349376 Pbyte
